# Implementing TF-IDF without using Scikit-Learn Library

<font face='georgia'>
    <h4><strong>Introduction:</strong></h4>

Typically, the tf-idf weight is composed by two terms: the first computes the normalized Term Frequency (TF), aka. the number of times a word appears in a document, divided by the total number of words in that document; the second term is the Inverse Document Frequency (IDF), computed as the logarithm of the number of the documents in the corpus divided by the number of documents where the specific term appears.

 <ul>
    <li>
<strong>TF:</strong> Term Frequency, which measures how frequently a term occurs in a document. Since every document is different in length, it is possible that a term would appear much more times in long documents than shorter ones. Thus, the term frequency is often divided by the document length (aka. the total number of terms in the document) as a way of normalization: <br>

$TF(t) = \frac{\text{Number of times term t appears in a document}}{\text{Total number of terms in the document}}.$
</li>
<li>
<strong>IDF:</strong> Inverse Document Frequency, which measures how important a term is. While computing TF, all terms are considered equally important. However it is known that certain terms, such as "is", "of", and "that", may appear a lot of times but have little importance. Thus we need to weigh down the frequent terms while scale up the rare ones, by computing the following: <br>

$IDF(t) = \log_{e}\frac{\text{Total  number of documents}} {\text{Number of documents with term t in it}}.$
for numerical stabiltiy we will be changing this formula little bit
$IDF(t) = \log_{e}\frac{\text{Total  number of documents}} {\text{Number of documents with term t in it}+1}.$
</li>
</ul>

<br>
<h4><strong>Example</strong></h4>
<p>

Consider a document containing 100 words wherein the word cat appears 3 times. The term frequency (i.e., tf) for cat is then (3 / 100) = 0.03. Now, assume we have 10 million documents and the word cat appears in one thousand of these. Then, the inverse document frequency (i.e., idf) is calculated as log(10,000,000 / 1,000) = 4. Thus, the Tf-idf weight is the product of these quantities: 0.03 * 4 = 0.12.
</p>
</font>

## 1. Building a TFIDF Vectorizer & comparing its results with Sklearn.

### Corpus

In [2]:
## SkLearn# Collection of string documents

corpus = [
     'this is the first document',
     'this document is the second document',
     'and this is the third one',
     'is this the first document',
]

### SkLearn Implementation

Sklearn formula of idf is different from the standard textbook formula. Here the constant <strong>"1"</strong> is added to the numerator and denominator of the idf as if an extra document was seen containing every term in the collection exactly once, which prevents zero divisions.
            
 $IDF(t) = 1+\log_{e}\frac{1\text{ }+\text{ Total  number of documents in collection}} {1+\text{Number of documents with term t in it}}.$

In [3]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer()
vectorizer.fit(corpus)
skl_output = vectorizer.transform(corpus)

In [4]:
# sklearn feature names, they are sorted in alphabetic order by default.

print(vectorizer.get_feature_names())

['and', 'document', 'first', 'is', 'one', 'second', 'the', 'third', 'this']


In [5]:
# Here we will print the sklearn tfidf vectorizer idf values after applying the fit method
# After using the fit function on the corpus the vocab has 9 words in it, and each has its idf value.

print(vectorizer.idf_)

[1.91629073 1.22314355 1.51082562 1.         1.91629073 1.91629073
 1.         1.91629073 1.        ]


In [6]:
# shape of sklearn tfidf vectorizer output after applying transform method.

skl_output.shape

(4, 9)

In [7]:
# sklearn tfidf values for first line of the above corpus.
# Here the output is a sparse matrix

print(skl_output[0])

  (0, 8)	0.38408524091481483
  (0, 6)	0.38408524091481483
  (0, 3)	0.38408524091481483
  (0, 2)	0.5802858236844359
  (0, 1)	0.46979138557992045


In [8]:
# sklearn tfidf values for first line of the above corpus.
# To understand the output better, here we are converting the sparse output matrix to dense matrix and printing it.
# Notice that this output is normalized using L2 normalization. sklearn does this by default.

print(skl_output[0].toarray())

[[0.         0.46979139 0.58028582 0.38408524 0.         0.
  0.38408524 0.         0.38408524]]


### Custom Implementation

In [9]:
from collections import Counter
from tqdm import tqdm
from scipy.sparse import csr_matrix
import math
import operator
from sklearn.preprocessing import normalize
import numpy

def fit(dataset):
    unique_words = set()                                #defined the varaible as a set for preventing duplicates
    if isinstance(dataset,(list)): 
        for row in dataset:                             #checing for each row in a dataset
            for word in row.split():                    #splitting the selected row and checking for each word
                if len(word) < 2:                       #discarding the punctuatuations and 'a'
                    continue
                unique_words.add(word)                  #adding the word in this row to the set
                                                        #enumerating the dataset to store index along with the sorted word.
        vocab = { j:i for i,j in enumerate(sorted(list(unique_words)))}   
        return vocab
    else:
        print("you need to pass list of sentences")

print(fit(corpus))


{'and': 0, 'document': 1, 'first': 2, 'is': 3, 'one': 4, 'second': 5, 'the': 6, 'third': 7, 'this': 8}


In [5]:
def word_in_doc(dataset,word): #defining a function to check the number of documents/reviews in which a word appears
    count = 0                  #initialising the variable with zero
    for idx, row in enumerate(dataset): #enumerating each row in the dataset and iterating row-wise
        for i in row.split():  #splitting each row in a dataset into words
            if i == word:      #checking if the input word is equal to the words in this row
                count=count+1  #if matched then increase the count by one and index by one
                idx= idx+1     #so that if a word appears multiple times in a row the count is still one and then next row is considered.
                break
    return count


In [6]:
def transform(dataset,vocab): 
    rows = []
    columns = []
    values = []
    if isinstance(dataset, list):
        documents = len(dataset)   #storing the number of documents in a variable. required for computing IDF
        for idx, row in enumerate(tqdm(dataset)): #checking for each row in the dataset. idx here will be used in row[]
            
            for word in row.split(): #for each word in the row
                if len(word) < 2:    # discarding punctuations
                    continue
               
                col_index = vocab.get(word,-1) #retireving the dimension number of the word from vocab and assigning it to column index
               
                # we will check if its there in the vocabulary that we build in fit() function
                # dict.get() function will return the values, if the key doesn't exits it will return -1
               
                if col_index != -1:  #only for the words that exist in vocab
                    
                    rows.append(idx)  #storing the index of the word
                    columns.append(col_index) #storing the dimension of the word
                   
                    #computing idf using the below formula
                    value = 1 + math.log((1+documents)/(1+word_in_doc(dataset,word))) 
                    
                    values.append(value) #storing the idf values 
        
        X = csr_matrix((values,(rows,columns)),shape = (len(dataset),len(vocab)) )
        
        return normalize(X, norm='l2', axis=1, copy=True, return_norm=False)
        #return csr_matrix((values,(rows,columns)),shape = (len(dataset),len(vocab)) )

    else:
        print("you need to pass list of strings")

In [7]:
vocab = fit(corpus)
print(vocab.keys())
print(transform(corpus,vocab)[0].toarray())

100%|██████████| 4/4 [00:00<00:00, 2712.13it/s]

dict_keys(['and', 'document', 'first', 'is', 'one', 'second', 'the', 'third', 'this'])
[[0.         0.46979139 0.58028582 0.38408524 0.         0.
  0.38408524 0.         0.38408524]]


As we can see above the output exactly matches with that of scikit-learn.

## 2. Implementing max features functionality

<ul>
    <li>As a part of this task we will have to modify our fit and transform functions so that our vocab will contain only 50 terms with top idf scores.
    <br>
    <li>This task is similar to our previous task, just that here our vocabulary is limited to only top 50 features names based on their idf values.
    <br>
    <li>Here we are using a pickle file, with file name cleaned_strings. We will load the corpus from this file and use it as input for our tfidf vectorizer.
     <ol>


In [55]:
# Below is the code to load the cleaned_strings pickle file 
# provided at https://github.com/rishabh4745/Implementing_TF-IDF_without_Scikit-Learn/blob/main/cleaned_strings
# Here corpus is of list type

import pickle
with open('cleaned_strings', 'rb') as f:
    corpus = pickle.load(f)
    
# printing the length of the corpus loaded
print("Number of documents in corpus = ",len(corpus))
print(corpus)


Number of documents in corpus =  746
['slow moving aimless movie distressed drifting young man', 'not sure lost flat characters audience nearly half walked', 'attempting artiness black white clever camera angles movie disappointed became even ridiculous acting poor plot lines almost non existent', 'little music anything speak', 'best scene movie gerardo trying find song keeps running head', 'rest movie lacks art charm meaning emptiness works guess empty', 'wasted two hours', 'saw movie today thought good effort good messages kids', 'bit predictable', 'loved casting jimmy buffet science teacher', 'baby owls adorable', 'movie showed lot florida best made look appealing', 'songs best muppets hilarious', 'cool', 'right case movie delivers everything almost right face', 'average acting main person low budget clearly see', 'review long overdue since consider tale two sisters single greatest film ever made', 'put gem movie terms screenplay cinematography acting post production editing directi

In [71]:
def word_in_doc(dataset,word):    #defining a function to check the number of documents/reviews in which a word appears  
    count = 0
    for idx, row in enumerate(dataset):
        for i in row.split():
            #print(row,idx)
            if i == word:
                count=count+1
                idx= idx+1
                break
    return count

def fit(dataset):
    unique_words = set()
    documents = len(dataset) #computing length of the document. required for calculating idf.
    vocab = {} 
    if isinstance(dataset,(list)):

        # code for finding and storing the set of unique words present in a dataset.
        for row in dataset:   # for each row in a dataset
            for word in row.split():  # for every word in a dataset obtained by splitting a row
                if len(word) < 2:     # ignoring punctuations
                    continue
                unique_words.add(word)

        # code for coputing idf for each unique word present in a dataset        
        for i in unique_words:  #for every word in the unique set
            idf = (1 + math.log((1+documents)/(1+word_in_doc(dataset,i)))) #Calculating IDF using word_in_doc function
            vocab[i] = idf       
        
        #code for sorting the dictionary and storing only top 50 words with highest IDF
        top_50_vocab = dict(sorted(vocab.items(),key= operator.itemgetter(1), reverse=True)[:50])     #https://www.geeksforgeeks.org/python-n-largest-values-in-dictionary/
        return top_50_vocab
    else:
        print("you need to pass list of sentences")


{'gallon': 6.922918004572872, 'sinking': 6.922918004572872, 'builders': 6.922918004572872, 'design': 6.922918004572872, 'straw': 6.922918004572872, 'deadpan': 6.922918004572872, 'jennifer': 6.922918004572872, 'naked': 6.922918004572872, 'punish': 6.922918004572872, 'unnecessary': 6.922918004572872, 'general': 6.922918004572872, 'moved': 6.922918004572872, 'pans': 6.922918004572872, 'commented': 6.922918004572872, 'riz': 6.922918004572872, 'walk': 6.922918004572872, 'translating': 6.922918004572872, 'limitations': 6.922918004572872, 'sack': 6.922918004572872, 'howdy': 6.922918004572872, 'blandly': 6.922918004572872, 'biggest': 6.922918004572872, 'kudos': 6.922918004572872, 'critic': 6.922918004572872, 'within': 6.922918004572872, 'splendid': 6.922918004572872, 'jealousy': 6.922918004572872, 'jones': 6.922918004572872, 'dozen': 6.922918004572872, 'foreign': 6.922918004572872, 'poler': 6.922918004572872, 'preservation': 6.922918004572872, 'cheerfull': 6.922918004572872, 'mindblowing': 6.9

In [60]:
def transform(dataset,vocab):
    rows = []
    columns = []
    values = []
    
    #creating a dictionary 'b' for storing the dimension of each word along with the word itself
    b= {j:i for i,j in sorted(enumerate(vocab.keys()))}
    if isinstance(dataset, list):
        documents = len(dataset)

        for idx, row in enumerate(dataset):
            for word in row.split():
                if len(word) < 2:
                    continue
                value = vocab.get(word,-1) #fetching and storing IDF value for the word using get()
                col_index = b.get(word,-1) #fetching and storing the dimension of the word using get()
                if value != -1 and col_index != -1: 
                    rows.append(idx)           #storing the index of the column in row
                    columns.append(col_index)  #storing the dimension of the word in col
                    values.append(value)       #storing the idf value of the word in values

        X = csr_matrix((values,(rows,columns)),shape = (len(dataset),len(vocab)) )
        
        return normalize(X, norm='l2', axis=1, copy=True, return_norm=False)   #retuning the normalized values

    else:
        print("you need to pass list of strings")

In [61]:
vocab = fit(corpus)
print(vocab.keys())
print(transform(corpus,vocab)[0].toarray())

dict_keys(['gallon', 'sinking', 'builders', 'design', 'straw', 'deadpan', 'jennifer', 'naked', 'punish', 'unnecessary', 'general', 'moved', 'pans', 'commented', 'riz', 'walk', 'translating', 'limitations', 'sack', 'howdy', 'blandly', 'biggest', 'kudos', 'critic', 'within', 'splendid', 'jealousy', 'jones', 'dozen', 'foreign', 'poler', 'preservation', 'cheerfull', 'mindblowing', 'gake', 'hero', 'includes', 'logic', 'baby', 'babysitting', 'treasure', 'jason', 'ryans', 'bob', 'guards', 'mistakes', 'touch', 'imagine', 'cars', 'sublimely'])
[10, 19, 19, 19, 38, 56, 95, 107, 134, 135, 135, 135, 135, 176, 178, 193, 290, 325, 335, 345, 370, 421, 436, 437, 452, 458, 458, 459, 464, 472, 477, 486, 486, 546, 561, 572, 584, 603, 604, 613, 614, 619, 626, 633, 644, 644, 644, 693, 714, 723] [38, 28, 9, 2, 27, 18, 15, 31, 0, 37, 17, 43, 16, 7, 30, 42, 8, 20, 45, 4, 23, 26, 29, 35, 25, 41, 11, 32, 3, 49, 40, 13, 48, 12, 14, 6, 21, 36, 44, 46, 10, 5, 22, 24, 1, 47, 19, 33, 34, 39] [6.922918004572872, 6.92

Reference:
www.appliedaicourse.com